In [1]:
from pyspark.sql import SparkSession
import pandas as pd 
from pyspark.sql.functions import col,to_timestamp,sum

In [2]:
spark = SparkSession.builder \
    .appName("EnergyForecastTransformation") \
    .master("local[*]")\
    .getOrCreate()
print('The connection is successfully stablished')

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/04/14 07:34:37 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


The connection is successfully stablished


In [3]:
spark_df = spark.read.option("header",True).csv("/home/sinatavakoli284/energy-forecast-pipeline/data/raw_data.csv")
pandas_df = pd.read_csv('/home/sinatavakoli284/energy-forecast-pipeline/data/raw_data.csv')

In [4]:
spark_df = spark_df.select(
    "utc_timestamp",
    "cet_cest_timestamp",
    "DE_load_actual_entsoe_transparency",
    "DE_load_forecast_entsoe_transparency",
    "DE_solar_capacity",
    "DE_solar_generation_actual",
    "DE_solar_profile",
    "DE_wind_capacity",
    "DE_wind_generation_actual",
    "DE_wind_profile",
    "DE_wind_offshore_capacity",
    "DE_wind_offshore_generation_actual",
    "DE_wind_offshore_profile",
    "DE_wind_onshore_capacity",
    "DE_wind_onshore_generation_actual",
    "DE_wind_onshore_profile",
    "DE_50hertz_load_actual_entsoe_transparency",
    "DE_50hertz_load_forecast_entsoe_transparency",
    "DE_50hertz_solar_generation_actual",
    "DE_50hertz_wind_generation_actual",
    "DE_50hertz_wind_offshore_generation_actual",
    "DE_50hertz_wind_onshore_generation_actual",
    "DE_LU_load_actual_entsoe_transparency",
    "DE_LU_load_forecast_entsoe_transparency",
    "DE_LU_price_day_ahead",
    "DE_LU_solar_generation_actual",
    "DE_LU_wind_generation_actual",
    "DE_LU_wind_offshore_generation_actual",
    "DE_LU_wind_onshore_generation_actual",
    "DE_amprion_load_actual_entsoe_transparency",
    "DE_amprion_load_forecast_entsoe_transparency",
    "DE_amprion_solar_generation_actual",
    "DE_amprion_wind_onshore_generation_actual",
    "DE_tennet_load_actual_entsoe_transparency",
    "DE_tennet_load_forecast_entsoe_transparency",
    "DE_tennet_solar_generation_actual",
    "DE_tennet_wind_generation_actual",
    "DE_tennet_wind_offshore_generation_actual",
    "DE_tennet_wind_onshore_generation_actual",
    "DE_transnetbw_load_actual_entsoe_transparency",
    "DE_transnetbw_load_forecast_entsoe_transparency",
    "DE_transnetbw_solar_generation_actual",
    "DE_transnetbw_wind_onshore_generation_actual"
)

In [5]:
spark_df.printSchema()

root
 |-- utc_timestamp: string (nullable = true)
 |-- cet_cest_timestamp: string (nullable = true)
 |-- DE_load_actual_entsoe_transparency: string (nullable = true)
 |-- DE_load_forecast_entsoe_transparency: string (nullable = true)
 |-- DE_solar_capacity: string (nullable = true)
 |-- DE_solar_generation_actual: string (nullable = true)
 |-- DE_solar_profile: string (nullable = true)
 |-- DE_wind_capacity: string (nullable = true)
 |-- DE_wind_generation_actual: string (nullable = true)
 |-- DE_wind_profile: string (nullable = true)
 |-- DE_wind_offshore_capacity: string (nullable = true)
 |-- DE_wind_offshore_generation_actual: string (nullable = true)
 |-- DE_wind_offshore_profile: string (nullable = true)
 |-- DE_wind_onshore_capacity: string (nullable = true)
 |-- DE_wind_onshore_generation_actual: string (nullable = true)
 |-- DE_wind_onshore_profile: string (nullable = true)
 |-- DE_50hertz_load_actual_entsoe_transparency: string (nullable = true)
 |-- DE_50hertz_load_forecast_

In [6]:
spark_df = spark_df.withColumn("utc_timestamp",to_timestamp("utc_timestamp"))
spark_df = spark_df.withColumn("cet_cest_timestamp",to_timestamp("cet_cest_timestamp"))

In [7]:
spark_df = spark_df.dropDuplicates(['utc_timestamp'])

In [8]:
spark_df = spark_df.select([col(i).alias(i.strip().lower().replace(" ","_")) for i in spark_df.columns])

In [9]:
numeric_col = [c for c in spark_df.columns if c not in ("utc_timestamp","cet_cest_timestamp")]
for column_name in numeric_col:
    spark_df = spark_df.withColumn(column_name,col(column_name).cast('double'))

In [10]:
spark_df.printSchema()

root
 |-- utc_timestamp: timestamp (nullable = true)
 |-- cet_cest_timestamp: timestamp (nullable = true)
 |-- de_load_actual_entsoe_transparency: double (nullable = true)
 |-- de_load_forecast_entsoe_transparency: double (nullable = true)
 |-- de_solar_capacity: double (nullable = true)
 |-- de_solar_generation_actual: double (nullable = true)
 |-- de_solar_profile: double (nullable = true)
 |-- de_wind_capacity: double (nullable = true)
 |-- de_wind_generation_actual: double (nullable = true)
 |-- de_wind_profile: double (nullable = true)
 |-- de_wind_offshore_capacity: double (nullable = true)
 |-- de_wind_offshore_generation_actual: double (nullable = true)
 |-- de_wind_offshore_profile: double (nullable = true)
 |-- de_wind_onshore_capacity: double (nullable = true)
 |-- de_wind_onshore_generation_actual: double (nullable = true)
 |-- de_wind_onshore_profile: double (nullable = true)
 |-- de_50hertz_load_actual_entsoe_transparency: double (nullable = true)
 |-- de_50hertz_load_for

In [11]:
null_threshold = 0.5
total_rows = spark_df.count()
cols_to_drop = [c for c in spark_df.columns if spark_df.filter(col(c).isNull()).count() / total_rows > null_threshold]
spark_df = spark_df.drop(*cols_to_drop)

In [12]:
spark_df.printSchema()

root
 |-- utc_timestamp: timestamp (nullable = true)
 |-- cet_cest_timestamp: timestamp (nullable = true)
 |-- de_load_actual_entsoe_transparency: double (nullable = true)
 |-- de_load_forecast_entsoe_transparency: double (nullable = true)
 |-- de_solar_capacity: double (nullable = true)
 |-- de_solar_generation_actual: double (nullable = true)
 |-- de_solar_profile: double (nullable = true)
 |-- de_wind_capacity: double (nullable = true)
 |-- de_wind_generation_actual: double (nullable = true)
 |-- de_wind_profile: double (nullable = true)
 |-- de_wind_offshore_capacity: double (nullable = true)
 |-- de_wind_offshore_generation_actual: double (nullable = true)
 |-- de_wind_offshore_profile: double (nullable = true)
 |-- de_wind_onshore_capacity: double (nullable = true)
 |-- de_wind_onshore_generation_actual: double (nullable = true)
 |-- de_wind_onshore_profile: double (nullable = true)
 |-- de_50hertz_load_actual_entsoe_transparency: double (nullable = true)
 |-- de_50hertz_load_for

In [ ]:
spark_df.coalesce(1).write.mode('overwrite').option('header',True).csv('/home/sinatavakoli284/energy-forecast-pipeline/data/clean_data.csv')